In [1]:
import pandas as pd
import defusedxml.ElementTree as ET
import re

## Školská zařízení (MŠ, ZŠ, střední aj.) - scrap kapacit

In [2]:
tree = ET.parse(r'..\data\open_data_MSMT\vrejcelk.xml')
root = tree.getroot()

In [3]:
skoly_data = []
zarizeni_info_data = []
zarizeni_misto_data = []
skola_obor_data = []

for ps in root.findall('.//PravniSubjekt'):
    # TAB "skoly" a "zarizeni_info"
    ico = ps.find('ICO').text
    nazev = ps.find('Reditelstvi').find('RedPlnyNazev').text
    for sz in ps.find('SkolyZarizeni').findall('SkolaZarizeni'):
        izo = sz.find('IZO').text
        typZarizeni = sz.find('SkolaPlnyNazev').text
        zarizeniDruhTyp = sz.find('SkolaDruhTyp').text
        zarizeniKapacita = sz.find('SkolaKapacita').text
        kapacitaJednotka = sz.find('SkolaKapacitaJednotka').text
        jazyk = sz.find('SkolaJazyk').text

        raw_skoly = {'IZO' : izo,  
                    'ZarizeniTyp': typZarizeni, 
                    'Kapacita': zarizeniKapacita,
                    'KapacitaJednotka': kapacitaJednotka,           
                    }
        raw_zarizeni_info = {
                    'ICO': ico, 
                    'ReditelstviNazev' : nazev, 
                    'ZarizeniDruhTyp' : zarizeniDruhTyp, 
                    'jazyk' : jazyk, 
                    'IZO' : izo,
                    }
        skoly_data.append(raw_skoly)
        zarizeni_info_data.append(raw_zarizeni_info)

        # TAB "zarizeni_misto"
        for smvc in sz.find('SkolaMistaVykonuCinnosti').findall('SkolaMistoVykonuCinnosti'):
            druh = smvc.find('MistoDruhTyp').text # u středních škol např. rozlišení gymnázií, u MŠ specializace

            # extrahování PSČ
            def extract_psc(adresa_text):
                if adresa_text:
                    PSC_match = re.search(r'\d{3} ?\d{2}', adresa_text)
                    return PSC_match.group() if PSC_match else adresa_text
                return None
            PSC = extract_psc(smvc.find('MistoAdresa2').text)        
            PSC2 = extract_psc(smvc.find('MistoAdresa3').text)

                        
            raw_zarizeni_misto = {
                    'druh' : druh,
                    'adresa2': PSC,
                    'adresa3': PSC2,
                    'IZO' : izo,
                   }
            zarizeni_misto_data.append(raw_zarizeni_misto)
            
        # TAB "skola_obor"
        if sz.find('SkolaOboryVzdelani') is not None:
            for sov in sz.find('SkolaOboryVzdelani').findall('SkolaOborVzdelani'):
                oborNazev = sov.find('OborNazev').text
                delkaVzdelavani = sov.find('DelkaVzdelavani').text
                oborKapacita = sov.find('OborKapacita').text
                oborDobihajici = sov.find('OborDobihajici').text
                # print(oborNazev)
    
                raw_skola_obor = {
                    'OborNazev' : oborNazev,
                    'DelkaVzdelavani': delkaVzdelavani,
                    'OborKapacita' : oborKapacita,
                    'OborDobihajici' : oborDobihajici,
                    'IZO' : izo,
                   }
                skola_obor_data.append(raw_skola_obor)

### Školy (kapacita) data -> dataframe dle ERD

Převedení raw dat do dataframů, následné vytvoření sloupců 'PSC' a 'obec' a jejich vyplnění dle nadefinovaných funkcí.\
**makePsc2** -> pokud se ve sloupcích adresa2 a adresa3 nachází PSC, vyextrahuje se k pozdějšímu napárování krajů (přes okresy)\
**makeObec** -> jestliže jsme získali PSC ze sloupce adresa3, pak do obce se nám zapíše adresa2+PSC jako unikátní kombinace identifikátoru (adresa3 mohla být none i jiná str informace), pokud adresa3 nebo PSC nebylo vyplněno, do obce se nám zapíše adresa2, \
Chybějící hodnoty v PSC nahradíme nulou a převedeme hodnoty na int


In [4]:
skoly = pd.DataFrame(skoly_data)

zarizeni_info = pd.DataFrame(zarizeni_info_data)
zarizeni_info.insert(loc=0, column='ID', value=range(1, len(zarizeni_info_data) + 1))
    
zarizeni_misto = pd.DataFrame(zarizeni_misto_data)
zarizeni_misto.insert(loc=0, column='ID', value=range(1, len(zarizeni_misto) + 1))
    
skola_obor = pd.DataFrame(skola_obor_data)
skola_obor.insert(loc=0, column='ID', value=range(1, len(skola_obor_data) + 1))
  
def makePsc(row):
    adresa2 = row['adresa2']
    adresa3 = row['adresa3']
    if adresa3:
        psc_match = re.search(r'\d{3} ?\d{2}', adresa3)
        if psc_match:
            return psc_match.group().replace(' ', '')
    elif adresa2:
        psc_match = re.search(r'\d{3} ?\d{2}', adresa2)
        if psc_match:
            return psc_match.group().replace(' ', '')
            
def makeObec(row):
    PSC_obec = row['PSC']
    adresa2 = row['adresa2']
    adresa3 = row['adresa3']
    if PSC_obec:
        if adresa3:
            psc_match = re.search(r'\d{3} ?\d{2}', adresa3)
            if psc_match.group().replace(' ', '') == PSC_obec:
                return adresa2 + ',' + PSC_obec
        return adresa2
    else:
        return adresa2
        

zarizeni_misto['PSC'] = zarizeni_misto.apply(
    makePsc, axis=1) 

zarizeni_misto['obec'] = zarizeni_misto.apply(   
    makeObec, axis=1)

# null hodnoty v 'PSC'nahrazeny číslem 0, následně celý sloupe převeden na int
zarizeni_misto['PSC'] = zarizeni_misto['PSC'].fillna(0)
zarizeni_misto = zarizeni_misto.astype({'PSC':'int'})
# len(zarizeni_misto[zarizeni_misto['PSC'] == 0]) # kde chybí psč máme alespoň identifikátor obec
zarizeni_misto['IZO'].value_counts()

IZO
181064791    108
110029534     56
108012263     46
161100473     41
108027384     35
            ... 
181144841      1
181144913      1
181144921      1
181145057      1
181145227      1
Name: count, Length: 27188, dtype: int64

Předpříprava převodníku (z GitHubu) PSČ -> kraj

In [5]:
psc_okres = pd.read_csv(r'..\data\adresa\psc2okres.csv') # obsahuje duplicity, a celkem uniq 77 okresů
okres_kraj = pd.read_csv(r'..\data\adresa\okres2kraj.csv') # pouze unikátní hodnoty, míň okresu, než má psc okresů, 82 okresů
psc_okres = psc_okres.drop_duplicates() # dropnutí duplikátu z cca 17tis na cca 3tis záznamů

Spojení tabulek k převodu - PSČ-Kraj, spojení přes okres.\
Vyhledání problematických PSC, které figutují ve vícero krajích (může zapříčinit zkreslení výsledků)\
**make_kraj** -> do vytvořeného sloupce 'Kraj' napárujeme dle PSC kraj pouze tehdy, pokud nám PSC nefiguruje ve vícero krajích 


In [6]:
# vytvoreni nové tabulky -> dle PSČ zjistime pravděpodobně kraj
psc_kraj = okres_kraj.merge(psc_okres, how='right', on='Okres')

#kde není vyplněn kraj, přehodíme tam okres - jedná se např. o Hl. město Prahu, aj.
psc_kraj['Kraj'] = psc_kraj['Kraj'].fillna(psc_kraj['Okres'])

# len(psc_kraj['PSC'].unique()) #3007, 3165 je celkový počet, tzn. 158 PSČ celkem figuruje ve vícero krajích

# psc figurující ve vícero krajích 
psc_duplicates = psc_kraj.groupby('PSC')['Kraj'].nunique()

shared_psc = psc_duplicates[psc_duplicates > 1].index
shared_kraje = psc_kraj[psc_kraj['PSC'].isin(shared_psc)]
shared_kraje_set = set(shared_kraje['PSC'])
shared_kraje_set_df = pd.DataFrame(shared_kraje_set)
shared_kraje_set_df = shared_kraje_set_df.rename(columns={0: 'PSC'})

def makeKraj(row):
    zarizeni_psc = row['PSC']
    
    if zarizeni_psc not in shared_kraje_set_df['PSC'].values:
        matching_row = psc_kraj[psc_kraj['PSC'] == zarizeni_psc]

        if not matching_row.empty:
            return matching_row['Kraj'].values[0]       
    else:
        return None
        
zarizeni_misto['Kraj'] = zarizeni_misto.apply(
    makeKraj, axis=1) 

Filtrování zařízení bez přiřazených krajů, následná extrakce, prohnání výsledků AI, náhodná ruční kontrola výsledků, nahrání a následné přiřazení výsledků k zarizeni_misto.\
Jedná se o zařízení, které buďto vůbec nemají PSČ (ale mají jiné str informace) nebo PSČ mají, ale to je přiřazeno ke dvěma krajům.

In [8]:
zarizeni_misto_filtered = zarizeni_misto[zarizeni_misto['Kraj'].isnull()]
zarizeni_misto_filtered

uniq_obec = zarizeni_misto_filtered['obec'].unique() # snízení záznamů na 1/7
uniq_obec = pd.DataFrame(uniq_obec).rename(columns={0: 'Kraj'})
# uniq_obec.to_excel(r'output_files\zarizeni_misto_filtered1.xlsx') #následně hodnoty zkopírovány a vloženy do Copilotu (Gemini ani ChatGPT nefungovali)

# zarizeni_misto['Kraj'].value_counts()
# uniq_obec['Kraj'].value_counts()
# zarizeni_misto.info()

Import(spuštění scriptu) dohledaných a hlavně problematických dat pomocí MS Copilot, převedení na df, následné napárovnání na zarizeni_misto

In [9]:
%run ../data/adresa/kraj_copilot.py
location = locations
location_df = pd.DataFrame(list(location.items()), columns=['Location', 'Kraj'])
location_df

def make_missingKraj(row):
    zarizeni_obec = row['obec']
    zarizeni_kraj = row['Kraj']
    
    if pd.isnull(zarizeni_kraj):
        match = location_df[location_df['Location'] == zarizeni_obec]
        if not match.empty:
            return match['Kraj'].values[0]
    return zarizeni_kraj
   
zarizeni_misto['Kraj'] = zarizeni_misto.apply(
    make_missingKraj, axis=1) 

Závěrečná kontrola - zarizeni_misto nemá v kraji nulové hodnoty + ve výčtu uniq hodnot jsou pouze kraje. 

In [10]:
zarizeni_misto['Kraj'] = zarizeni_misto['Kraj'].replace(['Frýdek Místek', 'Třinec'], 'Moravskoslezský kraj')
zarizeni_misto[zarizeni_misto['Kraj'].isnull()] # finální ověření, že žádný kraj nechybí
zarizeni_misto['Kraj'].value_counts()
# zarizeni_misto.info()

Kraj
Středočeský kraj        5263
Jihomoravský kraj       4653
Moravskoslezský kraj    4140
Hlavní město Praha      3335
Olomoucký kraj          2797
Ústecký kraj            2645
Jihočeský kraj          2333
Královéhradecký kraj    2293
Zlínský kraj            2270
Kraj Vysočina           2173
Pardubický kraj         2027
Plzeňský kraj           1920
Liberecký kraj          1610
Karlovarský kraj        1074
Name: count, dtype: int64